In [1]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None

In [2]:
drg = pd.read_excel('msdrgToServiceLine.xlsx')
drgMap = dict(zip(drg['MS-DRG #'],drg['MS-DRG Service Line Description']))

In [3]:
df = pd.read_pickle('hcupData.pkl')
cases = df[['HOSP_NIS','DRG']]
cases['Cases'] = 1
hospitals = cases.groupby(['HOSP_NIS','DRG']).sum().reset_index().sort_values('Cases',ascending=False)
hospitals['SL'] = hospitals['DRG'].replace(drgMap)
hospitals = hospitals.groupby(['HOSP_NIS','SL']).sum().reset_index()
hospitals = hospitals[['HOSP_NIS','SL','Cases']]

In [4]:
cols = [f'DX{i}' for i in range (1,26)]
slDf = df[['HOSP_NIS','DRG']+cols]
slDf.loc[slDf.isin(['V4986']).any(axis=1),'DNR'] = 1  #2012 ICD 9 code for DNR
slDf.loc[slDf.isin(['V667']).any(axis=1),'PC'] = 1 #2012 ICD 9 code for palliative care

In [5]:
slEol = slDf[['HOSP_NIS','DRG','DNR','PC']]
slEol['SL'] = slEol['DRG'].replace(drgMap)
slEol = slEol.groupby(['HOSP_NIS','SL']).sum().reset_index()

In [8]:
eolDf = slEol.merge(hospitals,on=['HOSP_NIS','SL'])
# eolDf['%DNR'] = eolDf['DNR']/eolDf['Cases']*100
# eolDf['%PC'] = eolDf['PC']/eolDf['Cases']*100

In [11]:
eolDf[['HOSP_NIS','SL','Cases','DNR','PC']].to_excel('eolData.xlsx', index=False)